# Using and Building a Prediction Market

## Introduction

Prediction markets are marketplaces where participants trade **shares** of outcomes of a future event. These markets harness the wisdom of the crowd, **aggregating information** from many individuals into a collective prediction (often expressed as a probability or odds). A higher market price for an outcome implies a higher perceived probability of that outcome occurring, as traders buy and sell shares based on their knowledge and beliefs.

**Information aggregation:** In a prediction market, prices serve as signals. If someone has information that the current price/probability is wrong, they have an incentive to trade (buy underpriced outcomes or sell overpriced ones) to profit from their information. In doing so, they push the price toward a more accurate value. Markets thus act as information aggregators, combining disparate pieces of knowledge into one quantitative forecast&#8203;:contentReference[oaicite:0]{index=0}&#8203;:contentReference[oaicite:1]{index=1}.

**Incentive compatibility and scoring rules:** Designing a mechanism to elicit truthful probabilities from individuals is non-trivial. *Proper scoring rules* were one approach: they reward a forecaster based on the accuracy of the probability they report for an outcome, and are structured so that the forecaster’s best strategy is to report their true belief. For example, the Brier score and log score are proper scoring rules that pay more (or penalize less) when the reported probability is closer to the actual outcome (with maximum reward when you state the true probability). The downside is that proper scoring rules are typically one-shot and don’t easily aggregate multiple people’s opinions&#8203;:contentReference[oaicite:2]{index=2}.

**Market scoring rules and LMSR:** *Market scoring rules* (introduced by Hanson) extend this idea to a market setting. Instead of each person reporting a probability once, a market scoring rule allows a sequence of participants to each adjust a probability estimate, with each trader effectively scoring the previous trader’s prediction and then setting a new prediction. The last probability stands as the market’s consensus. Hanson’s **Logarithmic Market Scoring Rule (LMSR)** is a popular mechanism for automated prediction markets: it provides a formula for the market price based on outstanding shares, and guarantees that the market maker’s worst-case loss is bounded&#8203;:contentReference[oaicite:3]{index=3}&#8203;:contentReference[oaicite:4]{index=4}. The LMSR functions as an **automated market maker (AMM)**, always willing to take the other side of a trade. When traders buy shares of an outcome, the LMSR AMM increases the price of that outcome and decreases the price of others according to a fixed **cost function**. This ensures prices (implied probabilities) move in response to trades, reflecting the confidence behind those trades.

**Automated Market Makers vs. Order Books:** Traditional exchanges use order books (LOBs) where buyers and sellers place orders and trades occur when orders match. In a low-liquidity setting (like a niche prediction market on an obscure topic), there may not be enough both-side interest to form a market, whereas an AMM can provide continuous liquidity from a pool of funds or algorithm. However, AMMs often require the market creator to put up initial capital and can incur a loss (subsidy) to ensure traders are paid out. For widely-followed events with high volume (e.g. presidential elections or popular sports games), a **limit order book** may be more capital-efficient as liquidity comes from traders’ orders instead of a fixed pool&#8203;:contentReference[oaicite:5]{index=5}&#8203;:contentReference[oaicite:6]{index=6}. **Polymarket** is a platform that exemplifies a hybrid approach: for highly speculated binary events it uses an order book, while for lower-volume markets it offers an AMM. In summary, AMMs like LMSR make it easy to start a market and aggregate information with little initial traffic, whereas order books become preferable as market interest and volume grow.

---

In this lab, you will explore prediction markets both as a **participant** and as a **builder**:
- **Part 1: Using Manifold Markets** – You will create an account on a prediction market platform (Manifold Markets) and make a prediction in an actual market, observing how information might be reflected in prices.
- **Part 2: AMM Calculations** – You will solve a few short problems to solidify your understanding of how automated market makers (like LMSR and Uniswap’s constant product AMM) calculate prices and respond to trades.
- **Part 3: Coding an LMSR Market Maker** – You will examine a simplified Solidity smart contract that implements an LMSR-based prediction market for a binary outcome, with comments explaining how the mechanism works.

Let's get started!


## Part 1: Using Manifold Markets

Manifold Markets (https://manifold.markets) is a user-friendly, play-money prediction market platform. In Manifold, users trade in **M$ (Mana)**, a virtual currency, to bet on outcomes of yes/no questions, multiple-choice questions, or free-response questions. For this lab, we'll focus on yes/no markets (binary outcome markets) which act like the examples discussed in lecture.

**Step 1: Sign Up / Log In.** Go to **manifold.markets** and sign in (you can use a Google account for convenience). If you use a university email (e.g. your Princeton email), Manifold may credit you with some bonus Mana. New accounts generally start with a free allotment of M$ to enable trading.

**Step 2: Explore Markets.** Once logged in, take a moment to browse the available markets on the home page. You'll see questions along with a percentage (the current probability according to the market) and possibly a chart of recent movements. Click on a market that interests you to view its details. For instance, you might choose a market like "*Will Person X win the Y election?*". Inside a market, you can see the current probability, the history of trades, comments, and the interface to place a bet.

**Step 3: Join or Create a Market.** You can participate in an existing market by betting on an outcome, or even create your own market question. For now, pick an existing yes/no market that you find interesting. (If you have a specific question in mind, feel free to create a new market for it, but this is optional.) Note the market's current probability and think about what it implies.

**Step 4: Make a Prediction (Place a Bet).** In your chosen market, decide whether you want to bet on "Yes" or "No". You'll see a form where you can enter an amount of Mana to bet and it will show you the payout or new probability if you make that trade. Enter a small bet (e.g. M$10 or M$20) on the side you believe is more likely. Submit your bet. The market probability will update immediately after your trade. Observe how much it moved – if you bet a small amount on a very liquid market, the change might be tiny, whereas a larger bet or a less liquid market will shift the probability more noticeably.

**Step 5: Reflect on Information and Price Movements.** After making your prediction, consider what information could impact this market. What kinds of news, data releases, or events might cause the probability to change significantly? For example, if your market is on an election outcome, a new poll or a major news story could sway traders' opinions. If it's on a sports game, an injury or lineup change could affect the odds. 



## Part 2: AMM Calculation Exercises

Next, let's reinforce some concepts with a couple of calculation exercises. These will help you understand quantitatively how market makers set prices in prediction markets.

**1. LMSR Price Impact:** Suppose a prediction market uses the **LMSR (Log Market Scoring Rule)** automated market maker for a binary (yes/no) market. The liquidity parameter is $b = 10$ (a higher $b$ means more liquidity, so prices move less for a given trade). Initially, the market is at 50% (no shares have been traded yet). Now a trader buys 5 "Yes" shares. **Question:** Approximately how much Mana does the trader pay for those 5 shares, and what is the new implied probability of "Yes" after the trade?

*Hint:* The LMSR cost function for two outcomes is 
$$C(q_{yes}, q_{no}) = b \cdot \ln\!\Big(e^{q_{yes}/b} + e^{q_{no}/b}\Big),$$ 
where $q_{yes}$ and $q_{no}$ are the number of shares for each outcome in the market. The cost to buy shares is the increase in $C$. Initially $q_{yes}=q_{no}=0$. After buying 5 Yes shares, $q_{yes}=5, q_{no}=0$. Compute $C(5,0) - C(0,0)$ for the cost. The implied probability of Yes can be found by 
$$p_{yes} = \frac{e^{q_{yes}/b}}{e^{q_{yes}/b} + e^{q_{no}/b}}.$$ 
(You can use $e^{0.5} \approx 1.6487$ and $\ln 2 \approx 0.6931$ for calculations.)

**2. Constant-Product AMM:** Now consider a simpler constant-product market maker (like Uniswap) for a binary market. Imagine an AMM pool that starts with 50 "Yes" tokens and 50 "No" tokens. (Each Yes token will be worth USD 1 if the event outcome is Yes, and each No token worth USD 1 if the outcome is No.) The pool's invariant is $X_{yes} \cdot X_{no} = 2500$. Initially, the implied odds are 50%/50% because the reserves are equal. **Question:** If a trader buys 10 Yes tokens from this pool, how many No tokens do they have to put into the pool (i.e. how many No tokens does the AMM take from the trader)? What are the new reserves in the pool after the trade, and what is the new implied probability of Yes?

*Hint:* In a constant product AMM, the product of reserves stays constant. If the trader removes 10 Yes tokens from the pool, the new Yes reserve is $50 - 10 = 40$. To satisfy $X_{yes} \cdot X_{no} = 2500$, the new No reserve $X_{no}'$ must satisfy $40 \cdot X_{no}' = 2500$. Solve for $X_{no}'$. The price of Yes in terms of No (the exchange rate) is given by the ratio of No to Yes reserves. You can convert that exchange rate into an implied probability by comparing the relative values of Yes and No tokens.


## Part 3: Implementing an LMSR Market Maker in Solidity

Finally, let's examine how we could implement a prediction market as a smart contract. Below is a simplified Solidity contract for a binary outcome prediction market using the LMSR mechanism. The contract acts as an automated market maker:
- It allows users to buy "Yes" or "No" outcome shares by paying Ether (which acts as the currency).
- It keeps track of the total shares and uses the LMSR formula to determine the cost of shares (and thus the price).
- When the outcome is resolved (set by the owner of the contract), holders of the winning outcome shares can redeem 1 Ether per share (losing shares become worthless).

For simplicity, this contract uses Ether as the currency and does not use an external price oracle for resolution (we assume the owner will call the resolve function with the true outcome). In a real-world scenario, you'd want a decentralized way to resolve the market (for example, using a trusted oracle or a voting mechanism).

⚠️ **Note:** Implementing the LMSR cost function involves using exponentials and logarithms, which Solidity can't do with native types directly (at least not with full precision). In this code, the cost calculation is shown conceptually. In practice, you'd use a fixed-point math library or iterative approximation to calculate the cost and price. The focus here is on the structure and logic rather than the exact math implementation. We use the ABDK libraries to do this calculation.


```javascript
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.20;

// Import the ABDK Math library for fixed-point math functions (ln, exp, etc.)
import "abdk-libraries-solidity/ABDKMath64x64.sol";

contract LMSRPredictionMarket {
    using ABDKMath64x64 for int128;

    // Liquidity parameter for LMSR: higher = more liquidity, less price movement per trade
    uint256 public immutable b;
    address public owner;
    bool public marketResolved;
    bool public outcome; // true = Yes wins, false = No wins

    // Totals of shares purchased
    uint256 public totalYesShares;
    uint256 public totalNoShares;

    // Each user's Yes/No share balance
    mapping(address => uint256) public yesBalance;
    mapping(address => uint256) public noBalance;

    constructor(uint256 _b) payable {
        b = _b;
        owner = msg.sender;
    }

    modifier onlyOwner() {
        require(msg.sender == owner, "Not owner");
        _;
    }

    // Calculate LMSR cost for a trade using exponential scoring rule
    function calculateCost(
        uint256 qYesBefore,
        uint256 qNoBefore,
        uint256 qYesAfter,
        uint256 qNoAfter
    ) public view returns (uint256) {
        int128 b64 = ABDKMath64x64.fromUInt(b);

        // Convert quantities to fixed-point format and divide by b
        int128 qYes1 = ABDKMath64x64.fromUInt(qYesBefore).div(b64);
        int128 qNo1 = ABDKMath64x64.fromUInt(qNoBefore).div(b64);
        int128 qYes2 = ABDKMath64x64.fromUInt(qYesAfter).div(b64);
        int128 qNo2 = ABDKMath64x64.fromUInt(qNoAfter).div(b64);

        // Compute cost before and after the trade
        int128 cost1 = (qYes1.exp().add(qNo1.exp())).ln();
        int128 cost2 = (qYes2.exp().add(qNo2.exp())).ln();

        // Final cost = b * (cost2 - cost1)
        int128 costDelta = (cost2.sub(cost1)).mul(b64);
        return costDelta.toUInt();
    }

    // Buy Yes shares and pay Ether based on cost function
    function buyYes(uint256 amount) external payable {
        require(!marketResolved, "Market resolved");

        // Calculate cost of buying additional Yes shares
        uint256 cost = calculateCost(
            // TODO
        );

        require(msg.value >= cost, "Insufficient payment");

        // TODO: Update user balance and total shares
        

        // TODO: Refund any excess ETH sent
        
    }

    // Buy No shares and pay Ether based on cost function
    function buyNo(uint256 amount) external payable {
        require(!marketResolved, "Market resolved");

        // Calculate cost of buying additional No shares
        uint256 cost = calculateCost(
            // TODO
        );

        require(msg.value >= cost, "Insufficient payment");

        // TODO: Update user balance and total shares
        

        // TODO: Refund any excess ETH sent
    }

    // Resolve the market with the actual outcome (only callable by owner)
    function resolveMarket(bool _outcome) external onlyOwner {
        require(!marketResolved, "Already resolved");
        marketResolved = true;
        outcome = _outcome;
    }

    // Redeem winning shares for ETH (1 share = 1 ETH payout)
    function redeem() external {
        require(marketResolved, "Market not resolved");

        if (outcome) {
            // TODO: If "Yes" wins
            
        } else {
            // TODO: If "No" wins
            
        }
    }
}
```

This contract demonstrates the core logic of an LMSR-based prediction market. Notice that the **pricing mechanism** (the LMSR cost function) automatically adjusts the price based on how many shares of each side have been bought. If many "Yes" shares are purchased, the cost to buy additional Yes shares increases (and equivalently the implied probability of "Yes" goes up). The contract must hold enough Ether to pay out all winning shares; the parameter $b$ and any initial funding determine the maximum liability (worst-case payout minus collected fees).

By completing this lab, you have:
- Explored a live prediction market and seen how market probabilities reflect traders’ information.
- Practiced calculating price changes in two types of automated market makers.
- Reviewed how a smart contract can implement a prediction market mechanism.

Prediction markets continue to be an exciting area where finance, crowdsourced information, and blockchain technology intersect. We hope this lab gave you insight into how these markets function under the hood and how they can aggregate information in a decentralized way.